<a href="https://colab.research.google.com/github/PedroAMtz/Multi-Class-Prediction-of-Obesity-Risk/blob/main/Models_Experiments_Obesity_Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Models Experiments - Baseline + Fine-Tuning + Hyperparameter exploration 🤖🎯

In this Notebook we will focus on models experiments in depth, reaching a baseline model and then fine-tuning for obtaining better performance. The main idea is to implement different classification models from sickit-learn module, explore fine-tuning and understand generally each of the hyperparameter from each model proposed. With this, the objective is to build some relevant theoretical + practical knowledge about ML algorithms and use the Obesity Risk Multi-class prediction dataset to practice.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_path = "/content/drive/MyDrive/Data_Science_Foundations/playground-series-s4e2/"

train_data = pd.read_csv(data_path + "train.csv")
train_data.head()